In [ ]:
!pip install ultralytics supervision opencv-python numpy

In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO
import os

# --- CẤU HÌNH ---
# Hãy đảm bảo file video nằm đúng đường dẫn này
VIDEO_PATH = "data/processed_videos/ngatu_video.mp4" 
MODEL_PATH = "yolov8s.pt"

# Tạo thư mục output nếu muốn lưu video kết quả (Tùy chọn)
os.makedirs("output", exist_ok=True)
OUTPUT_PATH = "output/result.mp4"

# --- CẤU HÌNH VẠCH ĐẾM ---
# Lưu ý: Tọa độ này phải khớp với độ phân giải video của bạn (ví dụ 640x640)
LINE_START = sv.Point(300, 500)
LINE_END = sv.Point(600, 500)

# 'in' hoặc 'out' (Chạy thử để xác định chiều mong muốn)
TARGET_DIRECTION = 'out'

In [ ]:
def send_to_kafka(cam_id, vehicle_id, direction):
    """
    Hàm giả lập gửi dữ liệu đi. 
    Sau này bạn thay bằng producer.send()
    """
    print(f"🚀 [KAFKA SEND] Cam: {cam_id} | Xe #{vehicle_id} | Hướng: {direction} -> SUCCESS")

In [ ]:
# 1. Khởi tạo Model & Tracker
print("⏳ Đang tải model...")
model = YOLO(MODEL_PATH)
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)

# Track buffer cao giúp giữ ID lâu hơn
tracker = sv.ByteTrack(frame_rate=video_info.fps, track_buffer=60)

# 2. Khởi tạo LineZone & Set khử trùng
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)
counted_ids = set()

# 3. Annotators
box_annotator = sv.BoxAnnotator(thickness=2)
# Chỉnh scale text nhỏ lại cho dễ nhìn
line_zone_annotator = sv.LineZoneAnnotator(thickness=2, text_thickness=1, text_scale=0.5)

# Generator đọc video
frame_generator = sv.get_video_frames_generator(VIDEO_PATH)

print(f"✅ Bắt đầu xử lý video: {VIDEO_PATH}")
print(f"🎯 Chiều đếm mục tiêu: {TARGET_DIRECTION}")
print("Nhấn 'q' trên cửa sổ video để thoát hoặc nhấn nút Stop trên Notebook.")

try:
    for frame in frame_generator:
        # --- A. PHÁT HIỆN & TRACKING ---
        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        
        # Lọc class: Car(2), Motor(3), Bus(5), Truck(7)
        detections = detections[np.isin(detections.class_id, [2, 3, 5, 7])]
        detections = tracker.update_with_detections(detections)

        # --- B. XỬ LÝ LOGIC ĐẾM ---
        cross_in, cross_out = line_zone.trigger(detections)

        # [CÁCH 1]: Lọc hướng
        if TARGET_DIRECTION == 'in':
            trigger_bools = cross_in
        else:
            trigger_bools = cross_out

        if np.any(trigger_bools):
            triggering_ids = detections.tracker_id[trigger_bools]
            for vehicle_id in triggering_ids:
                # [CÁCH 2]: Khử trùng
                if vehicle_id not in counted_ids:
                    counted_ids.add(vehicle_id)
                    send_to_kafka("CAM_01", vehicle_id, TARGET_DIRECTION)

        # --- C. VẼ HÌNH ---
        # Vẽ line zone trước
        line_zone_annotator.annotate(frame, line_counter=line_zone)
        # Vẽ box xe sau
        box_annotator.annotate(scene=frame, detections=detections)

        # Hiển thị số lượng Kafka sent
        cv2.putText(frame, f"Kafka Sent: {len(counted_ids)}", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # --- D. HIỂN THỊ (QUAN TRỌNG) ---
        # Nếu chạy trên Google Colab, hãy bỏ comment 2 dòng dưới và comment dòng cv2.imshow
        # from google.colab.patches import cv2_imshow
        # cv2_imshow(frame) 
        
        # Chạy trên máy local:
        cv2.imshow("Traffic Counter", frame)

        # Nhấn 'q' để thoát sớm
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Đã dừng bởi người dùng.")
            break

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

finally:
    # Luôn dọn dẹp tài nguyên dù code có lỗi hay bị stop
    print("Đang đóng video và giải phóng bộ nhớ...")
    cv2.destroyAllWindows()
    print("Hoàn tất.")